# NLP 실습 #3: N-gram Language model
황순원의 '소나기' 로 trigram language model 학습하고 사용해보기
</br>
</br>

* 준비 : konlpy와 nltk의 tokenizer 설명

* Step 1: Data preprocessing - tokenization

* Step 2: Probability Table을 생성

* Step 3: Probability Table을 활용하여 시퀀스 확률 찾기

* Step 4: 주어진 시퀀스에서 다음 형태소 예측

* Step 5: 주어진 시퀀스로 시작하는 문장을 예측


</br>
</br>

In [2]:
from collections import defaultdict
import math
from konlpy.tag import Kkma
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

ModuleNotFoundError: No module named 'konlpy'

## 준비: konlpy의 형태소 분석기와 nltk의 tokenizer

In [1]:
# 1. konlpy의 Kkma 형태소 분석기 import, 형태소 분석 예제

tokenizer = Kkma()
tokenizer.morphs("이 문장이 형태소 단위로 잘 출력되나요?")

ModuleNotFoundError: No module named 'konlpy'

In [ ]:
# 이전에 nltk 및 'punkt' 모듈을 다운로드 하지 않은 경우에 주석을 풀고 run

sent_tokenize('입력이 문장 단위로 잘 구분되는지 확인해보세요. sent_tokenize는 한 line을 받아서 마침표를 기준으로 여러 문장으로 분리해주는 함수입니다! 이 문장은 세번째 문장입니다. 이 문장은 네번째 문장입니다!')

## Step 1: Data preprocessing - tokenization
&emsp;&emsp; sonagi.doc의 내용을 문장 단위로 분리.

</br> &emsp;&emsp; <<Exercise 1>> 각 문장을 kkma 분석기로 형태소 단위로 분리.


In [ ]:
# 4. nltk import하고 sonagi.doc을 읽어들임. nltk의 sent_tokenize를 이용하여 sonagi.doc의 내용을 문장 단위로 분리

from nltk.tokenize import sent_tokenize

lines = []
with open('sonagi.doc', 'rt', encoding='utf-8') as f:
    for line in f:
        lines.append(line)

sentences = []
for line in lines:
    sentences.extend(sent_tokenize(line))

In [ ]:
print(sentences)

In [ ]:
#########################################################################################################################
# 5. <<Exercise 1>> 주어진 sentences를 이용하여, sentences를 각 문장별로 형태소 분석한 morph_sentences를 implementation 해주세요.

# sentences: ['소년은 개울가에서 소녀를 보자 곧 윤 초시네 증손녀(曾孫女)딸이라는 걸 알 수 있었다.',
#             '소녀는 개울에다 손을 잠그고 물장난을 하고 있는 것이다.',
#             ... ]
#########################################################################################################################


morphs_lst = []
for sentence in sentences:
    temp = tokenizer.morphs(sentence)
    morphs_lst.append(temp)


# <expected results>
# morphs_lst: [['소년', '은', '개울가', '에서', '소녀', '를', '보', '자', '곧', '윤', '초시', '네', '증손녀', '(', '曾孫女', ')', '딸', '이', '라는', '것', '을', '알', 'ㄹ', '수', '있', '었', '다', '.'],
#         ['소녀', '는', '개울', '에다', '손', '을', '잠그', '고', '물장난', '을', '하', '고', '있', '는', '것', '이', '다', '.'],
#                   ... ]

In [ ]:
# 6. 형태소 분석된 결과물 확인

print(sentences[0])
print(morphs_lst[0])
print()
print(sentences[1])
print(morphs_lst[1])

## Step 2 Probability Table을 생성
&emsp;&emsp; << Exercise 1 >> 형태소 단위로 분리된 각 문장들의 trigram을 찾아서 리스트로 저장

</br> &emsp;&emsp; << Exercise 2 >> 리스트로 저장된 trigram을 종류 별로 count하여 dictionary 형의 probability table을 생성


In [ ]:
# << Exercise 1 >>
# Goal: 형태소 단위로 분리된 각 문장들의 trigram을 찾아서 리스트로 저장.
# 
# input: morphs- 한 문장의 형태소들의 list.
#         ex) [pad, <s>, 소녀, 는, 개울, 에다, 손, 을, 잠그, 고, 물장난, 을, 하, 고, 있, 는, 것, 이, 다, . </s>, pad]
# output: trigrams - 입력 문장의 모든 trigram을 tuple로 표현한 list.
#         ex) [((pad, <s>), 소녀), ((<s>, 소녀), 는), ((소녀, 는), 개울), ((는, 개울), 에다), ((개울, 에다), 손) ... ... ((이, 다), .), ((다, .), </s>), ((., </s>), pad),

def find_trigram(morphs):
    
    trigrams = [((t0, t1,), t2) for t0, t1, t2 in zip(morphs, morphs[1:], morphs[2:])]
    
    return trigrams


print(find_trigram(morphs_lst[1]))
print()
print(find_trigram(['pad', '<s>']  + morphs_lst[1]+ ['</s>','pad']))

In [ ]:
# << Exercise 2 >>
# Goal: 형태소 분석된 전체 문서의 trigram들을 count하여 parsing table을 dictionary 형으로 저장.
#       위의 find_trigram 함수를 활용할 것.
# input : morphs_lst
#        [['소년', '은', '개울가', '에서', '소녀', '를', '보', '자', '곧', '윤', '초시', '네', '증손녀', '(', '曾孫女', ')', '딸', '이', '라는', '것', '을', '알', 'ㄹ', '수', '있', '었', '다', '.'],
#         ['소녀', '는', '개울', '에다', '손', '을', '잠그', '고', '물장난', '을', '하', '고', '있', '는', '것', '이', '다', '.'],
#                   ... ]
# output : trigram_freq
#        {(pad, <s>): {소년: 25, 소녀: 35, 서울: 3, 벌써: 1, ...},
#         (<s>, 소년): {은: 18, 이: 7, ...},
#           ...
#         (소년, 은): {개울가: 1, 개울: 1, 저: 3, 이: 1, 조약돌: 1, 전: 1, ...}}

def count_trigram(morphs_lst):
    trigram_freq = defaultdict(lambda: defaultdict(int))
    for morphs in morphs_lst:
        trigram_in_a_sentence = find_trigram(['pad', '<s>']  + morphs + ['</s>','pad'])
        for trigram in trigram_in_a_sentence:
            trigram_freq[trigram[0]][trigram[1]] += 1
    return trigram_freq

trigram_freq = count_trigram(morphs_lst)
print(trigram_freq[('소년', '은')])

## Step 3: model을 활용하여 주어진 시퀀스 확률 구하기
&emsp;&emsp; << Exercise 2 >> 생성한 probability table을 사용하여 문장 '단풍 이 눈 에 따갑 없 다 .' 의 확률을 구하기

In [ ]:
# << Exercise 3 >>
# Goal: '단풍 이 눈 에 따갑 었 다 .' 의 확률 구하기
# input: target_trigram - [((pad, <s>), 단풍), ((<s>, 단풍), 이), ((단풍, 이), 눈), ((아, 눈), 에), ... (., </s>, pad)]
# output: probability - 0.0013549039433771487

target = "pad <s> 단풍 이 눈 에 따갑 었 다 . </s> pad".split()
target_trigram = find_trigram(target)
print(target_trigram)

def find_score(target_trigram, trigram_freq):
    prob = 1
    for trigram in target_trigram:
        numerator = 0
        denominator = 0
        for unigram, count in trigram_freq.get(trigram[0], {}).items():
            if unigram == trigram[1]:
                numerator = count
            denominator += count
            
        if denominator == 0:
            temp_prob = 0.
        else:
            temp_prob = numerator / denominator
            
        prob *= temp_prob
    return prob

find_score(target_trigram, trigram_freq)

## Step 4: 주어진 시퀀스에서 다음 형태소 예측

### &emsp;<< Exercise 2>> '어쩐지' 다음에 올 가장 자연스러운 형태소를 찾아주세요.

In [ ]:
# << Exercise 4 >> 
# Goal: "어쩐지" 다음에 올 형태소로 가장 적합한 것은?
# input: target - [pad, <s>, 소녀, 가]
#        trigram_freq - {(pad, <s>): {소년: 25, 소녀: 35, 서울: 3, 벌써: 1, ...},
#                        (<s>, 소년): {은: 18, 이: 7, ...},
#                            ...
#                        (소년, 은): {개울가: 1, 개울: 1, 저: 3, 이: 1, 조약돌: 1, 전: 1, ...}}
# output: best_token - '앉'

target = "pad <s> 어쩐지".split()

def find_next_token(target, trigram_freq):
    highest_count = 0
    best_token = None
    for unigram, count in trigram_freq.get((target[-2], target[-1]), {}).items():
        if highest_count < count:
            highest_count = count
            best_token = unigram

    return best_token
print(find_next_token(target, trigram_freq))

## Step 5: 주어진 시퀀스로 시작하는 가장 자연스러운 문장 예측

#### </br>&emsp;<< Exercise 3 >> Step 4와 loop문을 사용하여,
#### </br> &emsp;'어쩐지' 으로 시작하는 가장 자연스러운 문장을 찾아주세요.

In [ ]:
# << Exercise 5 >>
# Goal: "어쩐지"로 시작하는 가장 자연스러운 문장을 찾아주세요.
# input: target - [pad, <s>, 소녀, 가]
#        trigram_freq - {(pad, <s>): {소년: 25, 소녀: 35, 서울: 3, 벌써: 1, ...},
#                        (<s>, 소년): {은: 18, 이: 7, ...},
#                            ...
#                        (소년, 은): {개울가: 1, 개울: 1, 저: 3, 이: 1, 조약돌: 1, 전: 1, ...}}
# output: full_sentence - [어쩐지, 소녀, 의, 그림자, 가, 뵈, 지, 않, 았, 다, ., </s>]

def fine_full_sentence(partial_sentence, trigram_freq)
    loop_condition = True
    full_sentence= partial_sentence

    while (loop_condition):
        next_token = find_next_token(partial_sentence.split(), trigram_freq)
        partial_sentence += " " + next_token
        if partial_sentence.split()[-1] == '</s>':
            loop_condition = False
            
    return full_sentence
print(fine_full_sentence(target, trigram_freq))